In [2]:
import pandas as pd
import numpy as np

In [71]:
import requests
from bs4 import BeautifulSoup

In [139]:
from bs4 import BeautifulSoup 
import requests                
import sys


username = "/Ncarango9 "                               
url = "http://www.twitter.com/" + username

response = None

try:
    response = requests.get(url)
except Exception as e:
    print(repr(e))
    sys.exit(1)

if response.status_code != 200:
    print("Non success status code returned "+str(response.status_code))
    sys.exit(1)

soup = BeautifulSoup(response.text, 'html.parser')

if soup.find("div", {"class": "errorpage-topbar"}):
    print("\n\n Error: Invalid username.")
    sys.exit(1)

tweets = soup.find_all("p", {"class": "TweetTextSize TweetTextSize--normal js-tweet-text tweet-text"})

tweetList = []
for tweet in tweets:
    text = tweet.text.encode('utf-8')
    tweetList.append(text)

for tweetText in tweetList:
    print(tweetText)
    print("\n")

Non success status code returned 400


SystemExit: 1

/Users/rishabhsharma/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [149]:
url = "https://twitter.com/Ncarango9"

try:
    response = requests.get(url)
except Exception as e:
    print(repr(e))
    sys.exit(1)


In [150]:
soup = BeautifulSoup(response.text, 'html.parser')

In [151]:
tweets = soup.find_all("p")